# District catigorization on various demographic indicators
The goal of this project is to creata more accurate benchmarks for school district preformance. When comparing the preformance of school districts often state or neigboring school districts are used as benchmarks; even when socio-dconomic factors can be extreamly diffrent. Some notable factors that I think may cause districts to preform diffrently is teacher compensation,  teacher student ethnic distributions (speficcily ethnic homogeneity), 

as stronger homoginininty would corilate to stronger outcomes using chorpis christi and the RGV as an example where high student teacher ethnic hogoininity shows high student preformane and outcomes when looking at STAAR or college redinesss scores in hispanic communities leading to high student preformance.



In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Steps
1. Create a data set of all the district refrences for n number of years (I want to see if I can get at least 5 years back)
2. I have to clean them all and then actally make one master list of districts that have been around since 2022 to 2025. Partially because I don't want to clean. but also I think it doens't matter because the change will be so tiny.




Notes--
columns I need to keep
- District_Num = DISTRICT = 6 Digit County District Number	
- District_Name = DISTNAME	= District Name	
- County_Num = COUNTY = 3 Digit County Number
- County_Name = CNTYNAME = County Name
- ESC = REGION = 2 Digit ESC Region	
- ESC_Name = REGNNAME = Region Name	
- Charter = DFLCHART = District 2025 Flag = Charter Operator (Y/N)
- AEA = DFLALTED = District Rated Under AEA Procedures Flag
- District_Rating = D_RATING = 2025 District Overall Grade   -------------------- Pull Score too
- DAD_Post = 2025 District Post Secondary Distinction	
- ASVAB_Status = ASVAB Status PEIMS Description Short
- OUTCOME = District 2025 Special Education Determination Status



## Clean and create Master Districts Dataset

```district_master``` - This is a cleaned up list of school districts pulled together from 2013 & 2022-2025 years of data, keeping only the districts and fields that exist in all 5 years i.e. district ID, district name, county, region, and a couple of simple flags like whether it’s a charter so I can quickly see what each district.

***To-Dos / Notes***
- Re-title columns
- I will eventually need to re-clean this because rn It very bare bones
- I need to consider charter is not separated
- for all incoming data I will need to remove districts not in my district_final_cut 
- and if I go back more than 2022 if one of my districts in the final cut disappears then I need to remove them from the consideration to.

i.e. 
a district is not listed for any number of reasons before 2019 but in my current list of districts from 2022-2025 if i used data back to 2014 then it will skew... actually I just loaded in 2012 and added that to the set cross refrence and it narrowd down the list a little more so im just going with those ones. 




In [ ]:
# Load in District data
district_2025 = pd.read_csv(r'District Infromation\2025 District Reference.csv')
district_2024 = pd.read_csv(r'District Infromation\2024 District Reference.csv')
district_2023 = pd.read_csv(r'District Infromation\2023 District Reference Information.csv')
district_2022 = pd.read_csv(r'District Infromation\2022 District Reference Information, Accountability Rating and Special Education Determination Status.csv')
district_2013 = pd.read_csv(r'District Infromation/2013 District Reference .dat', dtype=str)

In [ ]:
district_25_headers = district_2025.iloc[0]
district_25 = district_2025[1:]
district_25.columns = district_25_headers
district_25.insert(loc=0, column="Year", value="2025")

district_25.head(1)

In [ ]:
district_2024_headers = district_2024.iloc[0]
district_24 = district_2024[1:]
district_24.columns = district_2024_headers
district_24.insert(loc=0, column="Year", value="2024")

district_24.head(1)

In [ ]:
district_2013.insert(loc=0, column="Yeas", value="2012")
district_2022.insert(loc=0, column="Year", value="2022")
district_2023.insert(loc=0, column="Year", value="2023")

district_22 = district_2022
district_23 = district_2023

In [ ]:
d_25 = set(district_25['DISTRICT'])
d_24 = set(district_24['DISTRICT'])
d_23 = set(district_23['DISTRICT'])
d_22 = set(district_22['DISTRICT'])
d_12 = set(district_2013['DISTRICT'])

# Creating a set of districts only in all
District_final_cut = d_25 & d_24 & d_23 & d_22 & d_12
District_final_cut
len(District_final_cut)


In [ ]:
# Check district lenths arent changing  
# 2025: Original List Len: 1208 New List Len: 1208
# 2024: Original List Len: 1207 New List Len: 1207
# 2023: Original List Len: 1209 New List Len: 1209
# 2022: Original List Len: 1207 New List Len: 1207

print(f'Original List Len: {len(district_22['DISTRICT'])} New Set Len: {len(d_22)}')


In [ ]:
def remove_districts(districts):
    filter = districts['DISTRICT'].isin(District_final_cut)
    filtered_district = districts[filter]
    return filtered_district

In [ ]:
district_25_filtered = remove_districts(district_25)
district_24_filtered = remove_districts(district_24)
district_23_filtered = remove_districts(district_23)
district_22_filtered = remove_districts(district_22)
district_13_filtered = remove_districts(district_2013)


In [ ]:
dist_25_col = set(district_25_filtered.columns)
dist_24_col = set(district_24_filtered.columns)
dist_23_col = set(district_23_filtered.columns)
dist_22_col = set(district_22_filtered.columns)
dist_13_col = set(district_2013.columns)

dist_col_in_all = dist_25_col & dist_24_col & dist_23_col & dist_22_col & dist_13_col
dist_col_in_all

In [ ]:
# now I need to make a new data fram that is a version of all of these where all the data is the same, and I want to see what wrong with the ones that arent.
# i.e. if a cdn is around an but the district name is diffrent or county etc

dist_df_list = [district_25_filtered, district_24_filtered, district_23_filtered, district_22_filtered, district_13_filtered]

districts_grouped = pd.concat(
    [df.loc[:, df.columns.intersection(dist_col_in_all)] for df in dist_df_list]   
)

districts_grouped

In [ ]:
# Check how many unique 'charter_status' values each district has
charter_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

aea_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

# IDs where the status changed (more than 1 unique value)
# inconsistent_ids = charter_check[charter_check > 1].index  # Nothing
inconsistent_ids = aea_check[aea_check > 1].index
inconsistent_ids

In [ ]:
districts_grouped.shape

districts_cleaned = districts_grouped[~districts_grouped['DISTRICT'].isin(inconsistent_ids)]

print(districts_grouped.shape, districts_cleaned.shape)


In [ ]:
# using 2025 as source of truth

district_master = districts_cleaned[districts_cleaned["Year"]=='2025'].reset_index(drop=True)
district_master = district_master.drop(columns='Year')


district_master.head()

## Clean Student & Staff Datasets

```student_master``` - 

***To-Dos / Notes***

- Account for any data masking (haven't handled that yet)
- Add testing and validation throughout the process to validate each step.
- Clean up the keys table, to account for columns that are the same thing but named differently depending on the yea, handle fields that only exist in some years but are still useful.

***Useful links***
- View had data is masked: https://rptsvr1.tea.texas.gov/perfreport/tapr/2025/masking.html
- 2013 data download headers explained: https://rptsvr1.tea.texas.gov/perfreport/tapr/2013/download/taprref.html

In [ ]:
# Load in all staff data
StudentStaff_Info_2013 = pd.read_csv(r"Staff & Student Data\2013 Staff & Student Information.txt")
StudentStaff_Info_2014 = pd.read_csv(r"Staff & Student Data\2014 Staff & Student Information.dat")
StudentStaff_Info_2015 = pd.read_csv(r"Staff & Student Data\2015 Staff & Student Information.dat")
StudentStaff_Info_2016 = pd.read_csv(r"Staff & Student Data\2016 District Staff & Student Information.dat")
StudentStaff_Info_2017 = pd.read_csv(r"Staff & Student Data\2017 Staff & Student Information.dat")
StudentStaff_Info_2018 = pd.read_csv(r"Staff & Student Data\2018 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2019 = pd.read_csv(r"Staff & Student Data\2019 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2020 = pd.read_csv(r"Staff & Student Data\2020 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2021 = pd.read_csv(r"Staff & Student Data\2021 Staff, Student, and Annual Graduates.csv")
StudentStaff_Info_2022 = pd.read_csv(r"Staff & Student Data\2022 Staff, Student, and Annual Graduates.csv")
StudentStaff_Info_2023 = pd.read_csv(r"Staff & Student Data\2023 Staff, Student, and Annual Graduates.csv")
Student_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Student Information.csv")
Staff_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Staff Information.csv")
Student_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Student Information.csv")
Staff_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Staff Information.csv")


In [ ]:
# Explore data
StudentStaff_Info_2014.columns

### Seprate Student and Staff Data 2013 - 2023

- When I downloaded the datasets for student and staff information 2013 and 2023 the student staff data was combined. I will want to separate these later but to make managing this many tables scalable I will throw then in a dict. ```Note: I will add 2024 and 2025 separated student and staff DataFrames to a dictionary later.```
- Load in the student staff key files

Future Goal: Combining all student and staff data downloads in master keys based on all unique rows but i think a function will be quicker for me to do now so I'll merge the headers later

In [ ]:
# Add student and staff 2013 and 2023 combined dataframes to a dictionary.

student_staff_dfs = {2013:StudentStaff_Info_2013, 
                      2014:StudentStaff_Info_2014, 
                      2015:StudentStaff_Info_2015, 
                      2016:StudentStaff_Info_2016, 
                      2017:StudentStaff_Info_2017,
                      2018:StudentStaff_Info_2018,
                      2019:StudentStaff_Info_2019, 
                      2020:StudentStaff_Info_2020,
                      2021:StudentStaff_Info_2021,
                      2022:StudentStaff_Info_2022,
                      2023:StudentStaff_Info_2023}
type(student_staff_dfs)

In [ ]:
# Loading in all the student and staff keys for 2013 to 2024
student_key_tables ={}
staff_key_tables = {}

for year in range(2013, 2024):
    # Load in files with a year variable
    student_key = pd.read_csv(rf"Staff & Student Keys\{year}_TAPR_Advanced_Data_Download_Student.csv")
    staff_key = pd.read_csv(rf"Staff & Student Keys\{year}_TAPR_Advanced_Data_Download_Staff.csv")

    student_key_tables[year] = student_key
    staff_key_tables[year] = staff_key

print(len(student_key_tables), len(staff_key_tables))


In [ ]:
# For the 2024 and 2025 school years, data sets had the header a human readable name I stripped that for now and promoted index 0 which used the old id naming conversion

# Note: I'll eventually need to save those header
def PromoteFirstRow(df, year):  
    new_headers = df.iloc[0]
    new_df = df.iloc[1:]
    new_df.columns = new_headers
    new_df = new_df.reset_index(drop=True)
    key_df = pd.DataFrame(new_headers)
    key_df = key_df.reset_index()
    key_df.columns = ["LABEL", "NAME"]

    return new_df, key_df

Student_Info_2024, key_df_stu_24 = PromoteFirstRow(Student_Info_2024, 2024)
student_key_tables[2024] = key_df_stu_24

Student_Info_2025, key_df_stu_25= PromoteFirstRow(Student_Info_2025, 2025)
student_key_tables[2025] = key_df_stu_25

Staff_Info_2024, key_df_staff_24= PromoteFirstRow(Staff_Info_2024, 2024)
staff_key_tables[2024] = key_df_staff_24

Staff_Info_2025, key_df_staff_25= PromoteFirstRow(Staff_Info_2025, 2025)
staff_key_tables[2025] = key_df_staff_25


In [ ]:
# Creating two dictionary one holding all the student data another holding all the staff data 

student_tables = {}
staff_tables = {}

for year in range(2013, 2024):
    # In the dictionary student_tables we have {year: DataFrame} i want to copy the data from the respective 


    student_tables[year] = student_staff_dfs[year][student_key_tables[year]['NAME']].copy()
    staff_tables[year] = student_staff_dfs[year][staff_key_tables[year]['NAME']].copy()

    student_tables[year].insert(loc=0, column="Year", value=year)
    staff_tables[year].insert(loc=0, column="Year", value=year)

    if year == 2023:

        
        student_tables[2024] = Student_Info_2024
        student_tables[2025] = Student_Info_2025
        student_tables[2024].insert(loc=0, column="Year", value=2024)
        student_tables[2025].insert(loc=0, column="Year", value=2025)

        staff_tables[2024] = Staff_Info_2024
        staff_tables[2025] = Staff_Info_2025
        staff_tables[2024].insert(loc=0, column="Year", value=2024)
        staff_tables[2025].insert(loc=0, column="Year", value=2025)



In [ ]:
# doing a basic test to see if the shapes make sense not sure why its plus one but not worries enough to check rn... i feel like it something to do with counting a header or index
for i in range(2013, 2024):    
    print(f"[{i}]: New Shape: {staff_tables[i].shape}, Expected shape: ({student_staff_dfs[i].shape[0]}, {staff_key_tables[i].shape[0]})")

#### Current Data State (Checkpoint)

As of now, the data has been manually cleaned and separated into two main structures:

- One dictionary containing **student dataframes**
- One dictionary containing **staff dataframes**

Each dictionary is keyed by **year**, covering **2013–2023**.

For both students and staff:
- Each dataframe represents a single year
- District is present and serves as the ID
- Year will be added or already exists as a column
- Data is mostly clean and usable for analysis

At this point:
- Student and staff data are fully separated
- No further loading or restructuring is needed

This is a stable checkpoint. Further refactoring or automation can be done later if needed.

In [ ]:
# Make a set/list of all the headers that exist across all the years data. Think District, Year, or columns like DPETWHIC. Doing this so i can Grab those columns later
staff_columns_by_year = {
    year: set(df.columns)
    for year, df in staff_tables.items()
    }

student_columns_by_year = {
    year: set(df.columns)
    for year, df in student_tables.items()
    }

# columns that exist in ALL years
staff_common_columns = set.intersection(*staff_columns_by_year.values())
student_common_columns = set.intersection(*student_columns_by_year.values())


In [ ]:
# Eventually I will need to clean the keys and add add any keys that either had small naming diff because of years or if I am ok with missing x numbers of years
# for something. like maybe they had chronic absenteeism on here at one point.

In [ ]:
# Make a new dict of data frames but make these data frames only ones with matching columns

staff_matching_col_tables = {}
student_matching_col_tables = {}

for year in staff_tables.keys():
    staff_matching_col_tables[year] = staff_tables[year][list(staff_common_columns)]

for year in student_tables.keys():
    student_matching_col_tables[year] = student_tables[year][list(student_common_columns)]

In [ ]:
staff_matching_col_tables[2025].head(2)

In [ ]:
# From the matching student and staff dataframes I am combing all of those into one long dataframe, one for student and one for staff
staff_dfs_list = []
student_dfs_list = []

for year, df in staff_matching_col_tables.items():
    staff_dfs_list.append(df)

for year, df in student_matching_col_tables.items():
    student_dfs_list.append(df)

staff_master = pd.concat(staff_dfs_list, ignore_index=True)
student_master = pd.concat(student_dfs_list, ignore_index=True)

In [ ]:
student_master.head(2)

In [ ]:
staff_master.head(2)